### Introduction
A model testing process was conducted using the BRATS2020 validation set. The validation set was downloaded and processed, and the resulting images were utilized for testing the model.

BRATS2020 validation set can find here: https://www.kaggle.com/datasets/awsaf49/brats20-dataset-training-validation

### Get packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import shutil

!git clone https://github.com/agustinroviraquezada/MRI_T1_T2_CycleGAN.git
requirements= "/content/MRI_T1_T2_CycleGAN/requirements.txt"
!pip install -r $requirements -q

Cloning into 'MRI_T1_T2_CycleGAN'...
remote: Enumerating objects: 1209, done.
remote: Counting objects: 100% (451/451), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 1209 (delta 356), reused 356 (delta 280), pack-reused 758
Receiving objects: 100% (1209/1209), 447.30 MiB | 43.58 MiB/s, done.
Resolving deltas: 100% (755/755), done.
Updating files: 100% (71/71), done.
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.0/330.0 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.2/308.2 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import nibabel as nib
import glob
import cv2
from torchvision.transforms.functional import normalize
import re
from torch.utils.data import Dataset, DataLoader
import torch
import os
import matplotlib.pyplot as plt
from torchmetrics import StructuralSimilarityIndexMeasure as SSIM
from torchmetrics import PeakSignalNoiseRatio as PSNR
import pandas as pd
import numpy as np
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from cycle.CycleGAN import CycleGAN
from cycle.DataMod import CycleGANDataModule,ImagePairTestSet
import random
import matplotlib.gridspec as gridspec
import imageio.v2 as imageio
from tqdm import tqdm
random.seed(478)

###Unzip data and prepare folders T1 and T2

In [ ]:
!unzip /content/drive/MyDrive/Brats_Data/BraTS2020_ValidationData.zip

Archive:  /content/drive/MyDrive/Brats_Data/BraTS2020_ValidationData.zip
caution: filename not matched:  /content/drive/MyDrive/Brats_Data/MICCAI_BraTS2020_ValidationData


In [ ]:
def PipeLine(file_path,destiny,indices):

    match = re.search(r"(\d+)_(t1|t2)\.nii", file_path)
    subject_number = match.group(1)


    center_slices=extract_center_slices(file_path,indices)
    crop_slice=crop_images(center_slices)
    ApplyOperations(crop_slice,destiny,subject_number,indices)



def extract_center_slices(file_path,slices):
    # Load the NIfTI file
    img = nib.load(file_path)
    data = img.get_fdata()

    # Extract the center slices
    center_slices=data[:, :, slices]
    rotdata=np.rot90(center_slices,k=3)
    ready_data=np.moveaxis(rotdata, 2, 0)
    return ready_data



def crop_images(data,margin=5,target_shape=(128, 128)):
    crop = []

    #Use a mask to find black margins and compute margings per each image
    for img in data:
        mask = img != 0
        row_mask = np.any(mask, axis=1)
        col_mask = np.any(mask, axis=0)

        rmin = np.maximum(np.argmax(row_mask) - margin, 0)
        rmax = row_mask.size - np.argmax(row_mask[::-1]) + margin
        cmin = np.maximum(np.argmax(col_mask) - margin, 0)
        cmax = col_mask.size - np.argmax(col_mask[::-1]) + margin

        rmax = min(rmax, img.shape[0] - 1)
        cmax = min(cmax, img.shape[1] - 1)

        #Crop the image
        cropped = img[rmin:rmax+1, cmin:cmax+1]
        resized_img = cv2.resize(cropped, target_shape, interpolation=cv2.INTER_LINEAR)
        crop.append(resized_img)

    if not crop:
        print("No images to stack. Please check the input data.")
        return np.array([])

    return np.stack(crop)


def ApplyOperations(img,destiny,subject_number,indices):
    #re-scale image to 0-1
    images_scaled = img / np.max(img, axis=(1, 2))[:, np.newaxis, np.newaxis]

    # Convert the numpy array to a PyTorch tensor
    img_tensor = torch.tensor(images_scaled, dtype=torch.float32).unsqueeze(1)

    # Normalize the tensor with mean=0.5 and std=0.5
    normalized_tensor = normalize(img_tensor, (0.5,), (0.5,))

    for i,s in zip(normalized_tensor,indices):
      torch.save(i,os.path.join(destiny,f"B_{subject_number}_{s}.pt"))

In [ ]:
rootFolder="/content/drive/MyDrive/Brats_Data/MICCAI_BraTS2020_ValidationData"
T1paths = glob.glob(rootFolder + "/**/*_t1.nii", recursive=True)
T2paths = glob.glob(rootFolder + "/**/*_t2.nii", recursive=True)

indices=[random.sample(range(70, 100), 5) for i in range(len(T1paths))]
for t1,idx in zip(T1paths,indices): PipeLine(t1,"/content/drive/MyDrive/Brats_Data/T1",idx)
for t2,idx in zip(T2paths,indices): PipeLine(t2,"/content/drive/MyDrive/Brats_Data/T2",idx)

### Define functions

In [ ]:
class ModelEvalBrats():
  def __init__(self,dataloader,device='cuda'):

    self.dataloader=dataloader
    self.device=device

  def RandomSamplePlot(self,ModelPath,mod):
    params = {'lr'        : 0.0002,
          'lbc_T1'        : 10,
          'lbc_T2'        : 10,
          'lbi'           : 0.1,
          'b1'            : 0.5,
          'b2'            : 0.999,
          'batch_size'    : 1,
          'im_channel'    : 1,
          'n_epochs'      : 9000,     #When it start. High number to not apply this
          'n_epochs_decay': 9000,     #Every each epoch to do High number to not apply this
          'mode'          : "linear",
          "target_shape"  : 1,
          "resnet_neck"   : 6,
          "features"      : 64}

    model2=CycleGAN(params)
    model2load=model2.load_from_checkpoint(checkpoint_path=ModelPath)

    # Instantiate your CycleGAN class (assuming it is already trained)
    model2load.eval()  # Set the model to evaluation mode
    model2load.to(self.device)  # Assuming you're using GPU
    ct=0
    T1_sample,T2_sample,T1f_sample,T2f_sample,T1C_sample,T2C_sample,T1I_sample,T2I_sample,names=[],[],[],[],[],[],[],[],[]
    T1_siimf,T1_ssimc,T1_ssimi,T2_siimf,T2_ssimc,T2_ssimi=[],[],[],[],[],[]
    ssim = SSIM().to(self.device)

    for T1,T2,T1_name,T2_name in self.dataloader:
      if random.random()>0.5:
        ct+=1
        T1 = T1.to(self.device)
        T2 = T2.to(self.device)

        T1_sample.append(torch.squeeze(T1.to("cpu")).numpy())
        T2_sample.append(torch.squeeze(T2.to("cpu")).numpy())

        match = re.search(r"B_(\d+)_(\d+)\.pt",T1_name[0])
        num1 = match.group(1)
        num2 = match.group(2)
        expression = f"S {int(num1)} S {int(num2)}"
        names.append(expression)



        # Generate a T2w image
        with torch.no_grad():  # Inference only
            T1_f=model2load.G_T2_T1(T2)
            T2_f=model2load.G_T1_T2(T1)

            T1_C=model2load.G_T2_T1(T2_f)
            T2_C=model2load.G_T1_T2(T1_f)

            T1_I=model2load.G_T2_T1(T1)
            T2_I=model2load.G_T1_T2(T2)



        T1f_sample.append(torch.squeeze(T1_f.to("cpu")).numpy())
        T2f_sample.append(torch.squeeze(T2_f.to("cpu")).numpy())
        T1C_sample.append(torch.squeeze(T1_C.to("cpu")).numpy())
        T2C_sample.append(torch.squeeze(T2_C.to("cpu")).numpy())
        T1I_sample.append(torch.squeeze(T1_I.to("cpu")).numpy())
        T2I_sample.append(torch.squeeze(T2_I.to("cpu")).numpy())


        T1_siimf.append(ssim(T1_f, T1).to("cpu").detach().item())
        T1_ssimc.append(ssim(T1_C, T1).to("cpu").detach().item())
        T1_ssimi.append(ssim(T1_I, T1).to("cpu").detach().item())
        T2_siimf.append(ssim(T2_f, T2).to("cpu").detach().item())
        T2_ssimc.append(ssim(T2_C, T2).to("cpu").detach().item())
        T2_ssimi.append(ssim(T2_I, T2).to("cpu").detach().item())



      if ct==5:
        break

    #T1 -->T2
    gs = gridspec.GridSpec(5, 5)  # Increase number of rows, one set of rows for images and one for histogram.
    fig = plt.figure(figsize=(10,12))  # Increase the height of the figure
    for t1,t2,t2f,t2c,t2i,sf,sc,si,ri,na in zip(T1_sample,T2_sample,T2f_sample,T2C_sample,T2I_sample,T2_siimf,T2_ssimc,T2_ssimi,range(6),names):

      ax1 = fig.add_subplot(gs[ri, 0])
      ax1.imshow(t1, cmap='gray')
      ax1.set_title(f'T1w {na}', size=10)
      ax1.axis('off')

      ax2 = fig.add_subplot(gs[ri, 1])
      ax2.imshow(t2, cmap='gray')
      ax2.set_title(f'T2w {na}', size=10)
      ax2.axis('off')

      ax3 = fig.add_subplot(gs[ri, 2])
      ax3.imshow(t2f, cmap='gray')
      ax3.set_title(f'sT2w SSIM:{sf:.2f}', size=10)
      ax3.axis('off')

      ax3 = fig.add_subplot(gs[ri, 3])
      ax3.imshow(t2c, cmap='gray')
      ax3.set_title(f'cT2w SSIM:{sc:.2f}', size=10)
      ax3.axis('off')

      ax4 = fig.add_subplot(gs[ri, 4])
      ax4.imshow(t2i, cmap='gray')
      ax4.set_title(f'iT2w SSIM:{si:.2f}', size=10)
      ax4.axis('off')

    plt.suptitle("Sample Real vs Generated T2")
    plt.tight_layout()
    plt.savefig(f"Plot_SampleEval_T2_Images_{mod}.svg", format='svg')
    plt.show()

    fig, axs = plt.subplots(5, 1, figsize=(10,12)) # Increase the height of the figure
    for t1,t2,t2f,t2c,t2i,sf,sc,si,ri,na in zip(T1_sample,T2_sample,T2f_sample,T2C_sample,T2I_sample,T2_siimf,T2_ssimc,T2_ssimi,range(6),names):
      axs[ri].hist(t2.flatten(), bins=100, density=True, histtype='step',facecolor='k',range=(-1,1),label="T2W")
      axs[ri].hist(t2f.flatten(), bins=100, density=True, histtype='step',facecolor='r',range=(-1,1),label="sT2W")
      axs[ri].hist(t2c.flatten(), bins=100, density=True, histtype='step',facecolor='b',range=(-1,1),label="cT2W")
      axs[ri].hist(t2i.flatten(), bins=100, density=True, histtype='step',facecolor='g',range=(-1,1),label="iT2W")

      axs[ri].set_xlim(-0.95,1)
      axs[ri].set_ylim(0,4)
      axs[ri].set_title(f'Intensity Distribution {na}')
      axs[ri].set_xlabel('Intensity', size=10)
      axs[ri].set_ylabel('Frequency', size=10)
      axs[ri].legend()

    plt.suptitle("Sample Real vs Generated T2")
    plt.tight_layout()
    plt.savefig(f"Plot_SampleEval_T2_Histogram_{mod}.svg", format='svg')
    plt.show()




    #T2 -->T1
    gs = gridspec.GridSpec(5, 5)  # Increase number of rows, one set of rows for images and one for histogram.
    fig = plt.figure(figsize=(10,12))  # Increase the height of the figure
    for t2,t1,t1f,t1c,t1i,sf,sc,si,ri,na in zip(T2_sample,T1_sample,T1f_sample,T1C_sample,T1I_sample,T1_siimf,T1_ssimc,T1_ssimi,range(6),names):

      ax1 = fig.add_subplot(gs[ri, 0])
      ax1.imshow(t2, cmap='gray')
      ax1.set_title(f'T2w {na}', size=10)
      ax1.axis('off')

      ax2 = fig.add_subplot(gs[ri, 1])
      ax2.imshow(t1, cmap='gray')
      ax2.set_title(f'T1w {na}', size=10)
      ax2.axis('off')

      ax3 = fig.add_subplot(gs[ri, 2])
      ax3.imshow(t1f, cmap='gray')
      ax3.set_title(f'sT1w SSIM:{sf:.2f}', size=10)
      ax3.axis('off')

      ax3 = fig.add_subplot(gs[ri, 3])
      ax3.imshow(t1c, cmap='gray')
      ax3.set_title(f'cT1w SSIM:{sc:.2f}', size=10)
      ax3.axis('off')

      ax4 = fig.add_subplot(gs[ri, 4])
      ax4.imshow(t1i, cmap='gray')
      ax4.set_title(f'iT1w SSIM:{si:.2f}', size=10)
      ax4.axis('off')

    plt.suptitle("Sample Real vs Generated T1")
    plt.tight_layout()
    plt.savefig(f"Plot_SampleEval_T1_Images_{mod}.svg", format='svg')
    plt.show()

    fig, axs = plt.subplots(5, 1, figsize=(10,12)) # Increase the height of the figure
    for t2,t1,t1f,t1c,t1i,sf,sc,si,ri,na in zip(T2_sample,T1_sample,T1f_sample,T1C_sample,T1I_sample,T1_siimf,T1_ssimc,T1_ssimi,range(6),names):
      axs[ri].hist(t1.flatten(), bins=100, density=True, histtype='step',facecolor='k',range=(-1,1),label="T2W")
      axs[ri].hist(t1f.flatten(), bins=100, density=True, histtype='step',facecolor='r',range=(-1,1),label="sT2W")
      axs[ri].hist(t1c.flatten(), bins=100, density=True, histtype='step',facecolor='b',range=(-1,1),label="cT2W")
      axs[ri].hist(t1i.flatten(), bins=100, density=True, histtype='step',facecolor='g',range=(-1,1),label="iT2W")

      axs[ri].set_xlim(-0.95,1)
      axs[ri].set_ylim(0,4)
      axs[ri].set_title(f'Intensity Distribution {na}')
      axs[ri].set_xlabel('Intensity', size=10)
      axs[ri].set_ylabel('Frequency', size=10)
      axs[ri].legend()

    plt.suptitle("Sample Real vs Generated T1")
    plt.tight_layout()
    plt.savefig(f"Plot_SampleEval_T1_Histogram_{mod}.svg", format='svg')
    plt.show()







  def GetMetrics(self,ModelPath):
    params = {'lr'        : 0.0002,
          'lbc_T1'        : 10,
          'lbc_T2'        : 10,
          'lbi'           : 0.1,
          'b1'            : 0.5,
          'b2'            : 0.999,
          'batch_size'    : 1,
          'im_channel'    : 1,
          'n_epochs'      : 9000,     #When it start. High number to not apply this
          'n_epochs_decay': 9000,     #Every each epoch to do High number to not apply this
          'mode'          : "linear",
          "target_shape"  : 1,
          "resnet_neck"   : 6,
          "features"      : 64}

    model2=CycleGAN(params)
    self.model2load=model2.load_from_checkpoint(checkpoint_path=ModelPath)


    # Instantiate your CycleGAN class (assuming it is already trained)
    self.model2load.eval()  # Set the model to evaluation mode
    self.model2load.to(self.device)  # Assuming you're using GPU
    Metrics=[]

    for T1,T2,T1_name,T2_name in tqdm(self.dataloader):
      T1_metics,T2_metrics=self.Compute_Metrics(T1,T2,T1_name,T2_name,ModelPath)
      Metrics.append(T1_metics)
      Metrics.append(T2_metrics)

    df=pd.DataFrame(Metrics,columns=["File","Modality","SSIM_C","SSIM_F","PSNR_C","PSNR_F","I_SSIM","I_PSNR"])
    result=df.groupby(['Modality']).agg(['mean','max', 'std']).reset_index()

    return result,df

  def Compute_Metrics(self,T1,T2,T1_name,T2_name,ModelPath):
    T1 = T1.to(self.device)
    T2 = T2.to(self.device)
    model=os.path.basename(ModelPath)

    # Generate a T2w image
    with torch.no_grad():  # Inference only
        f_T1 = self.model2load.G_T2_T1(T2)
        f_T2 = self.model2load.G_T1_T2(T1)

        C_T1=self.model2load.G_T2_T1(f_T2)
        C_T2=self.model2load.G_T1_T2(f_T2)

        I_T1=self.model2load.G_T2_T1(T1)
        I_T2=self.model2load.G_T1_T2(T2)

    #Define Metrics
    psnr= PSNR().to(self.device)
    ssim = SSIM().to(self.device)


    #SIIM
    C_T1_SSIM= ssim(C_T1, T1).to("cpu").detach().item()
    F_T1_SSIM= ssim(f_T1, T1).to("cpu").detach().item()
    I_T1_SSIM= ssim(I_T1, T1).to("cpu").detach().item()

    C_T2_SSIM= ssim(C_T2, T2).to("cpu").detach().item()
    F_T2_SSIM= ssim(f_T2, T2).to("cpu").detach().item()
    I_T2_SSIM= ssim(I_T2, T2).to("cpu").detach().item()


    #PNRS
    C_T1_PSNR= psnr(C_T1, T1).to("cpu").detach().item()
    F_T1_PSNR= psnr(f_T1, T1).to("cpu").detach().item()
    I_T1_PSNR= psnr(I_T1, T1).to("cpu").detach().item()

    C_T2_PSNR= psnr(C_T2, T2).to("cpu").detach().item()
    F_T2_PSNR= psnr(f_T2, T2).to("cpu").detach().item()
    I_T2_PSNR= psnr(I_T2, T2).to("cpu").detach().item()


    return (T1_name[0],"T1",C_T1_SSIM,F_T1_SSIM,C_T1_PSNR,F_T1_PSNR,I_T1_SSIM,I_T1_PSNR),(T2_name[0],"T2",C_T2_SSIM,F_T2_SSIM,C_T2_PSNR,F_T2_PSNR,I_T2_SSIM,I_T2_PSNR)

### Testing

In [ ]:
DataPath={"T1":"/content/drive/MyDrive/Brats_Data/T1","T2":"/content/drive/MyDrive/Brats_Data/T2"}
TestPath={"T1":"/content/drive/MyDrive/Brats_Data/T1","T2":"/content/drive/MyDrive/Brats_Data/T2"}
dataset=ImagePairTestSet(DataPath,TestPath,prb=0.09)
DataMRI_test = DataLoader(dataset, batch_size=1, shuffle=True)
Evaluacion=ModelEvalBrats(DataMRI_test,device='cuda')

In [ ]:
ModelPath="/content/MRI_T1_T2_CycleGAN/Models/BaseLine_model_0.694-276.ckpt"
Metrics,d_Baseline= Evaluacion.GetMetrics(ModelPath)
Metrics.to_csv("/content/MRI_T1_T2_CycleGAN/Models/MetricsBaseline_BestModel_informe.csv")

ModelPath="/content/MRI_T1_T2_CycleGAN/Models/Optimized_model_0.690-290.ckpt"
Evaluacion.RandomSamplePlot(ModelPath,"Baseline")